In [ ]:
import os
import sys
import math

from collections import Counter
import numpy as np 
import random

import torch
import torch.nn as nn
import torch.nn.functional as F 

# 这俩是nlp工具包
import nltk
import jieba

> 句子开头是BOS,结尾是EOS

In [ ]:
# 可自己定义,也可以直接定义dataset,loader,返回 (输入,标签)
def load_data(in_file):
    cn = []
    en = []
    num_examples = 0
    with open(in_file,'r') as f:
        for line in f:
            line = line.strip().split('\t') # 英文\t中文
            en.append(['BOS'] + nltk.word_tokenize(line[0].lower()) + ['EOS']) 
            # word_tokenize将英文句子拆成一个个单词,下面的中文cut也是一样的效果(拆成词语)
            cn.append(['BOS'] + list(jiesba.cut(line[1])) + ['EOS'])
    return en,cn

train_finle = 'nmt/en-cn/train.txt'
dev_file = 'nmt/en-cn/dev.txt'
train_en,train_cn = load_data(train_file)
dev_en,dev_cn = load_data(dev_file)

In [ ]:
UNK_IDX = 0
PAD_IDX = 1
def build_dict(sentences,max_words = 50000):
    word_count = Counter()
    for sentence in sentences:
        for s in sentence:
            word_count[s] += 1
    ls = word_count.most_common(max_words)
    total_words = len(ls) + 2

    word_dict = {w[0]: index + 2 for index,w in enumerate(ls)}
    word_dict['UNK'] = UNK_IDX
    word_dict['PAD'] = PAD_IDX
    # word_dict[word] == index
    # total_words = count
    return word_dict,total_words

In [ ]:
def encode(en_sentences,cn_sentences,en_dict,cn_dict,sort_by_len=True):
    length = len(en_sentences)
    out_en_sentences = [[en_dict.get(w,0) for w in sent] for sent in en_sentences]
    out_cn_sentences = [[cn_dict.get(w,0) for w in sent] for sent in en_sentences]

    def len_argsort(seq):
        return sorted(range(len(seq)),key=lamben x:len(seq[x]))
    if sort_by_len:
        sortex_index = len_argsort(out_en_sentences)
        out_en_sentences = [out_en_sentences[i] for i in sortex_index]
        out_cn_sentences = [out_cn_sentences[i] for i in sorted_index]
    return out_en_sentences,out_cn_sentences

In [ ]:
# 这个函数的作用是我们输入训练集的样本个数， batch_size大小， 就会返回多批 连续的batch_size个索引， 每一个索引代表一个样本
# 也就是可以根据这个索引去拿到一个个的batch
def get_minibatches(n, minibatch_size, shuffle=True):
    idx_list = np.arange(0, n, minibatch_size)
    if shuffle:
        np.random.shuffle(idx_list)
    minibatches = []
    for idx in idx_list:
        minibatches.append(np.arange(idx, min(idx+minibatch_size, n)))
    return minibatches      # 这个会返回多批连着的bath_size个索引  
#get_minibatches(len(train_en), 32)

# 这个函数是在做数据预处理， 由于每个句子都不是一样长， 所以通过这个函数就可以把句子进行补齐， 不够长的在句子后面添加0
def prepare_data(seqs):
    lengths = [len(seq) for seq in seqs]    # 得到每个句子的长度
    n_samples = len(seqs)       # 得到一共有多少个句子
    max_len = np.max(lengths)              # 找出最大的句子长度
    
    x = np.zeros((n_samples, max_len)).astype('int32')    # 按照最大句子长度生成全0矩阵
    x_lengths = np.array(lengths).astype('int32')
    for idx, seq in enumerate(seqs):        # 把有句子的位置填充进去
        x[idx, :lengths[idx]] = seq
    return x, x_lengths      # x_mask

def gen_examples(en_sentences, cn_sentences, batch_size):
    minibatches = get_minibatches(len(en_sentences), batch_size)   # 得到batch个索引
    all_ex = []
    for minibatch in minibatches:   # 每批数据的索引
        mb_en_sentences = [en_sentences[t] for t in minibatch]   # 取数据
        mb_cn_sentences = [cn_sentences[t] for t in minibatch]  # 取数据
        mb_x, mb_x_len = prepare_data(mb_en_sentences) # 填充成一样的长度， 但是要记录一下句子的真实长度， 这个在后面输入网络的时候得用
        mb_y, mb_y_len = prepare_data(mb_cn_sentences)
        all_ex.append((mb_x, mb_x_len, mb_y, mb_y_len))
    return all_ex

# batch_size = 64
# train_data = gen_examples(train_en, train_cn, batch_size)   # 产生训练集
# random.shuffle(train_data)
# dev_data = gen_examples(dev_en, dev_cn, batch_size)   # 产生验证集


GRU/LSTM的输入在进入时需要padding,但是处理时需要pack,不然必然有误差对吧

In [ ]:
def PlainEncoder(nn.Module):
    def __init__(self,vocab_size,hidden_size,dropout=0.2):
        super(PlainEncoder,self).__init__()
        self.embed = nn.Embedding(vocab_size,hidden_size)
        self.rnn = nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x,lengths):
        sorted_len,sorted_idx = lenghts.sort(0,descending=True)
        x_sorted = x[sorted_idx.long()]
        embedded = self.dropout(embed(x_sorted))
        packed_embedded = nn.utils.pack_padded_sequence(embedded, sorted_len.long().cpu().data.numpy(), batch_first=True) 

        packed_out,hid = self.rnn(packed_embedded)
        out ,_  = nn.utils.rnn.pad_packed_sequence(packed_out,batch_first=True)
        _,original_idx = sorted_idx.sort(0,descending=False)

        out = out[original_idx.long()].contiguous()  # contiguous是为了把不连续的内存单元连续起来 
        hid = hid[:, original_idx.long()].contiguous()
            
        return out, hid[[-1]]   # 把最后一层的hid给拿出来  这个具体看上面的简单演示

In [ ]:
# 这个基本上和Encoder是一致的， 无非就是初始化的h换成了Encoder之后的h
class PlainDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.2):
        super(PlainDecoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, y, y_lengths, hid):
        # y: [batch_size, seq_len-1]
        sorted_len, sorted_idx = y_lengths.sort(0, descending=True)    # 依然是句子从长到短排序
        y_sorted = y[sorted_idx.long()]
        hid = hid[:, sorted_idx.long()]
        
        y_sorted = self.dropout(self.embed(y_sorted)) # [batch_size, outpout_length, embed_size]
        
        pack_seq = nn.utils.rnn.pack_padded_sequence(y_sorted, sorted_len.long().cpu().data.numpy(), batch_first=True)
        out, hid = self.rnn(pack_seq, hid)   # 这个计算的是每个有效时间步单词的最后一层的隐藏状态
        unpacked, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)   # [batch, seq_len-1, hidden_size]
        _, original_idx = sorted_idx.sort(0, descending=False)
        output_seq = unpacked[original_idx.long()].contiguous()  # [batch, seq_len-1, hidden_size]
        
        hid = hid[:, original_idx.long()].contiguous()   # [1， batch, hidden_size]
        output = F.log_softmax(self.out(output_seq), -1)        
        # [batch, seq_len-1, vocab_size]   表示每个样本每个时间不长都有一个vocab_size的维度长度， 表示每个单词的概率
        
        return output, hid


In [ ]:
def PlainSeq2Seq(nn.Module):
    def __init__(self,encoder,decoder):
        super(PlainSeq2seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self,x,_lengths,y,y_lengths):
        encoder_out,hid = self.encoder(x,x_lengths)
        output,hid = self.decoder(y,y_lengths.hid)
        return output,None
    def translate(self,x,x_lengths,y,max_length=10):
        encoder_out,hid = self.encoder(x,x_lengths)
        preds = []
        batch_size = x.shape[0]
        attns = []
        for i in range(max_length):
            output,hid = self.decoder(y,torch.ones(batch_size).long().to(y.device),hid=hid)
            y = output.max(2)[1].view(batch_size,1)
            preds.append(y)
        return torch.cat(preds,1),None

In [ ]:
# masked cross entropy loss
class LanguageModelCriterion(nn.Module):
    def __init__(self):
        super(LanguageModelCriterion, self).__init__()
    
    def forward(self, input, target, mask):
        # input: [batch_size, seq_len, vocab_size]    每个单词的可能性
        input = input.contiguous().view(-1, input.size(2))   # [batch_size*seq_len-1, vocab_size]
        target = target.contiguous().view(-1, 1)    #  [batch_size*seq_len-1, 1]
        
        mask = mask.contiguous().view(-1, 1)   # [batch_size*seq_len-1, 1]
        output = -input.gather(1, target) * mask # 在每个vocab_size维度取正确单词的索引， 但是里面有很多是填充进去的， 所以mask去掉这些填充的
        # 这个其实在写一个NLloss ， 也就是sortmax的取负号
        output = torch.sum(output) / torch.sum(mask)
        
        return output  # [batch_size*seq_len-1, 1]